# Заполнятор досрочки

## Импорты

In [4]:
import time, os
start = time.time()

import pandas as pd
import docxtpl
import pymorphy2

from pytrovich.enums import NamePart, Gender, Case
from pytrovich.maker import PetrovichDeclinationMaker
from pytrovich.detector import PetrovichGenderDetector

morph = pymorphy2.MorphAnalyzer()
maker = PetrovichDeclinationMaker()
detector = PetrovichGenderDetector()

## Google-таблица

In [5]:
data = pd.read_csv('https://docs.google.com/spreadsheets/d/1c98xNkkaUL_lanW_yGBD8HRr69hrWzO-a2QjHs8fMyc/export?format=csv')

In [6]:
print(data.shape)
data

(63, 11)


,Отметка времени,Адрес электронной почты,Фамилия студента,Имя студента,Отчество студента,Учебная группа студента,Должность,Куда?,Фамилия руководителя подразделения,Имя руководителя подразделения,Отчество руководителя подразделения
0,18.03.2019 18:30:36,seermakov355@yandex.ru,Ермаков,Сергей,Романович,ИВМО-05-18,Директор,Институт ИТ,Зуев,Андрей,Сергеевич
1,18.03.2019 21:39:41,anna.ersak@mail.ru,Ерсак,Анна,Ярославовна,ЯЛНбо-14-17,Ректор,Институт Иностранных языков,Соколова,Наталья,Леонидовна
2,18.03.2019 21:51:08,nata7co@rambler.ru,Косяк,Наталья,Николаевна,ПДО-212,И.о. руководителя,"ГБПОУ г. Москвы ""Воробьевы Горы"" МКПТ",Жданов,Антон,Олегович
3,18.03.2019 21:52:19,caty-myu@yandex.ru,Петрова,Анастасия,Андреевна,ДС12,Директор,Автономная некомерческая организация высшего о...,Литовчин,Юрий,Михайлович
4,18.03.2019 22:03:32,ksu12g@mail.ru,Журавская,Ксения,Александровна,Ж-317,Ректор,Факультет liberal arts,Машин,Александр,Борисович
...,...,...,...,...,...,...,...,...,...,...,...
58,10.04.2019 17:32:47,vov4ok2000@yandex.ru,Вовчок,Сергей,Сергеевич,САП-211,Директору,Институт Пути Строительства и Сооружений,Шепитько,Таисия,Васильевна
59,11.04.2019 13:41:12,abdugafarov.olimkhon@gmail.com,Абдугафаров,Олимхон,Бахадир Угли,ЯЛНБО-01-18,Заведующему кафедрой,"Российский Университет Дружбы Народов, Институ...",Егорова,Людмила,Алексеевна
60,12.04.2019 10:43:38,sneginka-00@mail.ru,Борисова,Ульяна,Николаевна,ЭСБ-111,Директору,"МИИТ, институт экономики и финансов",Соколов,Юрий,Игоревич
61,12.04.2019 16:01:10,mkerlins@bk.ru,Мельникова,Кристина,Алексеевна,188,Декану,Институт филологии и истории,Хазанова,Маргарита,Игоревна


## Функции

In [7]:
def dative(row):
    dative_parser = morph.parse(row)[0]
    datv = dative_parser.inflect({'datv'})
    return datv.word

def genitive(row):
    words = row.split(' ')
    #print(words)
    final_list = []
    for word in words:
        gent_parser = morph.parse(word)[0]
        #print(gent_parser)
        gent = gent_parser.inflect({'gent'})
        #print(gent)
        final_list.append(gent)
    return final_list

def letter(row):
    return row[0] + '.'

def gender(df):
    genders = []
    for i in range(df.shape[0]):
        try:
            value = detector.detect(firstname=df['supervisor_name'][i], middlename=df['supervisor_patronymic'][i], lastname=df['supervisor_surname'][i])
            genders.append(value)
        except:
            value = 'NotPredicted'
            genders.append(value)
    return genders

def appeal(df):
    if df['supervisor_gender'] == Gender.FEMALE:
        return 'Уважаемая'
    else:
        return 'Уважаемый'

def dative_surname(df):
    if df['supervisor_gender'] == Gender.FEMALE:
        return maker.make(NamePart.LASTNAME, Gender.FEMALE, Case.DATIVE, df['supervisor_surname'])
    else: 
        return maker.make(NamePart.LASTNAME, Gender.MALE, Case.DATIVE, df['supervisor_surname'])

## Преобразования

In [8]:
transformed_data = pd.DataFrame(columns=['time', 'position', 'organization', 'supervisor_n1', 'supervisor_p1', 'supervisor_surname', 'dear', 'supervisor_name', 'supervisor_patronymic', 'student_fullname', 'student_group', 'supervisor_gender'])

transformed_data['time'] = data.iloc[:,0]
transformed_data['supervisor_n1'] = data.iloc[:,9].apply(letter).str.strip(' ')
transformed_data['supervisor_p1'] = data.iloc[:,10].apply(letter).str.strip(' ')
transformed_data['supervisor_surname'] = data.iloc[:,8].str.strip(' ')
transformed_data['supervisor_name'] = data.iloc[:,9].str.strip(' ')
transformed_data['supervisor_patronymic'] = data.iloc[:,10].str.strip(' ')
transformed_data['student_fullname'] = data.iloc[:,2].str.strip(' ') + ' ' + data.iloc[:,3].str.strip(' ') + ' ' + data.iloc[:,4].str.strip(' ')
transformed_data['student_group'] = data.iloc[:,5]
transformed_data['supervisor_gender'] = gender(transformed_data)
transformed_data['dear'] = transformed_data.apply(appeal, axis=1)

transformed_data['position'] = data.iloc[:,6].apply(dative).str.capitalize()
transformed_data['organization'] = "«" + data.iloc[:,7].str.rstrip(' ') + '»'#.apply(genitive)
transformed_data['supervisor_surname'] = transformed_data.apply(dative_surname, axis=1)

display(transformed_data)

,time,position,organization,supervisor_n1,supervisor_p1,supervisor_surname,dear,supervisor_name,supervisor_patronymic,student_fullname,student_group,supervisor_gender
0,18.03.2019 18:30:36,Директору,«Институт ИТ»,А.,С.,Зуеву,Уважаемый,Андрей,Сергеевич,Ермаков Сергей Романович,ИВМО-05-18,Gender.MALE
1,18.03.2019 21:39:41,Ректору,«Институт Иностранных языков»,Н.,Л.,Соколовой,Уважаемая,Наталья,Леонидовна,Ерсак Анна Ярославовна,ЯЛНбо-14-17,Gender.FEMALE
2,18.03.2019 21:51:08,И.о. руководителю,"«ГБПОУ г. Москвы ""Воробьевы Горы"" МКПТ»",А.,О.,Жданову,Уважаемый,Антон,Олегович,Косяк Наталья Николаевна,ПДО-212,Gender.MALE
3,18.03.2019 21:52:19,Директору,«Автономная некомерческая организация высшего ...,Ю.,М.,Литовчину,Уважаемый,Юрий,Михайлович,Петрова Анастасия Андреевна,ДС12,Gender.MALE
4,18.03.2019 22:03:32,Ректору,«Факультет liberal arts»,А.,Б.,Машину,Уважаемый,Александр,Борисович,Журавская Ксения Александровна,Ж-317,Gender.MALE
...,...,...,...,...,...,...,...,...,...,...,...,...
58,10.04.2019 17:32:47,Директору,«Институт Пути Строительства и Сооружений»,Т.,В.,Шепитько,Уважаемая,Таисия,Васильевна,Вовчок Сергей Сергеевич,САП-211,Gender.FEMALE
59,11.04.2019 13:41:12,Заведующему кафедре,"«Российский Университет Дружбы Народов, Инстит...",Л.,А.,Егоровой,Уважаемая,Людмила,Алексеевна,Абдугафаров Олимхон Бахадир Угли,ЯЛНБО-01-18,Gender.FEMALE
60,12.04.2019 10:43:38,Директору,"«МИИТ, институт экономики и финансов»",Ю.,И.,Соколову,Уважаемый,Юрий,Игоревич,Борисова Ульяна Николаевна,ЭСБ-111,Gender.MALE
61,12.04.2019 16:01:10,Декану,«Институт филологии и истории»,М.,И.,Хазановой,Уважаемая,Маргарита,Игоревна,Мельникова Кристина Алексеевна,188,Gender.FEMALE


In [9]:
transformed_data.to_csv(os.path.join('sources', 'test_csv.csv'))

In [10]:
values_dictionary = transformed_data.reset_index().to_dict('records')
#values_dictionary

In [11]:
test_dict = { 
    'position' : 'Ректор', 
    'organization' : 'РТУ МИРЭА', 
    'supervisor_n1': 'С.', 
    'supervisor_p1': 'А.', 
    'supervisor_surname': 'Кудж',
    'supervisor_name' : 'Станислав', 
    'supervisor_patronymic' : 'Алексеевич', 
    'student_fullname': 'Ермаков Сергей Романович', 
    'student_group': 'ИАМО-01-14',
    }

## Сохранение

In [12]:
def save_docs(values_dictionary):
    saved_docs = []
    for context in values_dictionary:
        doc = docxtpl.DocxTemplate(os.path.join('sources', 'prescheduled.docx'))
        doc.render(context)
        saved_doc = os.path.join('output', 'prescheduled', 'ДС_' + context['student_fullname'] + '.docx')
        doc.save(saved_doc)
        saved_docs.append(saved_doc)
    print(saved_docs)
    return saved_docs

In [13]:
save_docs(values_dictionary)

['output/prescheduled/ДС_Ермаков Сергей Романович.docx', 'output/prescheduled/ДС_Ерсак Анна Ярославовна.docx', 'output/prescheduled/ДС_Косяк Наталья Николаевна.docx', 'output/prescheduled/ДС_Петрова Анастасия Андреевна.docx', 'output/prescheduled/ДС_Журавская Ксения Александровна.docx', 'output/prescheduled/ДС_Иванишко Марина Юрьевна.docx', 'output/prescheduled/ДС_Комолов Сергей Алексеевич.docx', 'output/prescheduled/ДС_Чипко Александр Алексеевич.docx', 'output/prescheduled/ДС_Абрамова Анастасия Андреевна.docx', 'output/prescheduled/ДС_Киселева Виктория Дмитриевна.docx', 'output/prescheduled/ДС_Евтихов Павел Сергеевич.docx', 'output/prescheduled/ДС_Черняк Татьяна Олеговна.docx', 'output/prescheduled/ДС_Ныркова Валентина Игоревна.docx', 'output/prescheduled/ДС_Большенко Мария Анатольевна.docx', 'output/prescheduled/ДС_Метелёва Наталья Германовна.docx', 'output/prescheduled/ДС_Ерсак Анна Ярославовна.docx', 'output/prescheduled/ДС_Чермошенцева Анастасия Евгеньевна.docx', 'output/preschedu

['output/prescheduled/ДС_Ермаков Сергей Романович.docx',
 'output/prescheduled/ДС_Ерсак Анна Ярославовна.docx',
 'output/prescheduled/ДС_Косяк Наталья Николаевна.docx',
 'output/prescheduled/ДС_Петрова Анастасия Андреевна.docx',
 'output/prescheduled/ДС_Журавская Ксения Александровна.docx',
 'output/prescheduled/ДС_Иванишко Марина Юрьевна.docx',
 'output/prescheduled/ДС_Комолов Сергей Алексеевич.docx',
 'output/prescheduled/ДС_Чипко Александр Алексеевич.docx',
 'output/prescheduled/ДС_Абрамова Анастасия Андреевна.docx',
 'output/prescheduled/ДС_Киселева Виктория Дмитриевна.docx',
 'output/prescheduled/ДС_Евтихов Павел Сергеевич.docx',
 'output/prescheduled/ДС_Черняк Татьяна Олеговна.docx',
 'output/prescheduled/ДС_Ныркова Валентина Игоревна.docx',
 'output/prescheduled/ДС_Большенко Мария Анатольевна.docx',
 'output/prescheduled/ДС_Метелёва Наталья Германовна.docx',
 'output/prescheduled/ДС_Ерсак Анна Ярославовна.docx',
 'output/prescheduled/ДС_Чермошенцева Анастасия Евгеньевна.docx',
 

In [14]:
end = time.time()
print('Время работы скрипта: ', round(end - start, 2), 'секунд')

Время работы скрипта:  19.5 секунд


## Источники

https://habr.com/ru/post/456534/

https://blog.formpl.us/how-to-generate-word-documents-from-templates-using-python-cb039ea2c890

https://habr.com/ru/post/483302/

http://helpexe.ru/programmirovanie/kak-chitat-i-pisat-v-google-sheets-s-python

https://fooobar.com/questions/130030/download-a-spreadsheet-from-google-docs-using-python

https://ru.stackoverflow.com/questions/934988/%D0%9A%D0%B0%D0%BA-%D1%8F-%D0%BC%D0%BE%D0%B3%D1%83-%D0%B4%D0%BE%D0%B1%D1%8B%D1%82%D1%8C-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5-%D1%81-google-docs-%D1%81-%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E-python

https://issue.life/questions/44992650

https://ru.stackoverflow.com/questions/1157933/%D0%A1%D0%BA%D0%BB%D0%BE%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-%D1%81%D0%BB%D0%BE%D0%B2-%D0%BF%D0%BE-%D0%BF%D0%B0%D0%B4%D0%B5%D0%B6%D0%B0%D0%BC-%D0%B2-python

https://buildmedia.readthedocs.org/media/pdf/docxtpl/latest/docxtpl.pdf